In [7]:
import pandas as pd
import os
import shutil
import numpy as np
import glob
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

XML_folder='Gt_annotations/'
Pre_Files_dir='Predicted/'

File_name_change_Dir=XML_folder.replace('/','').replace('\\','')+'_INT_Fortmat'
Ground_truth_json=XML_folder.replace('/','').replace('\\','')+'.json'
Files_names_data_path=Pre_Files_dir.replace('/','').replace('\\','')+'.csv'

Files=os.listdir(XML_folder)
if not os.path.exists(File_name_change_Dir):
    os.mkdir(File_name_change_Dir)

Files_names_data=[]
for file in Files:
    shutil.copy(XML_folder+'/'+file,File_name_change_Dir+'/'+str(Files.index(file))+'.xml')
    Files_names_data.append([file,str(Files.index(file))+'.xml'])

Files_names_data=pd.DataFrame(Files_names_data,columns=['Old File Name','New File Name'])

In [8]:
from voc2coco import convert
xml_files = glob.glob(os.path.join(File_name_change_Dir, "*.xml"))
print("Number of xml files: {}".format(len(xml_files)))
convert(xml_files, Ground_truth_json)

Number of xml files: 2877


In [9]:
with open(Ground_truth_json,'r') as f:
    Data_gt=json.load(f)

Actual_Files_names=pd.DataFrame([[g['file_name'].split('\\')[-1],g['id']] for g in Data_gt['images']],columns=['Imagename','id'])
Pre_Files=os.listdir(Pre_Files_dir)

In [10]:

Data_predicted_json=[]
for ii in range(len(Pre_Files)):
#     print(ii)
    with open(Pre_Files_dir+'/'+Pre_Files[ii],'r') as f:
        prefile_data=f.read().split('\n')
        prefile_data=[g.split() for g in prefile_data if g!='']
        if len(prefile_data)==0:
            continue
        prefile_data=np.array(prefile_data)
        prefile_data=prefile_data.astype(object)
        prefile_data[:,1]=prefile_data[:,1].astype(float)
        prefile_data[:,2:]=prefile_data[:,2:].astype(int)
        
    File_new_name=Files_names_data[Files_names_data['Old File Name']==Pre_Files[ii].replace('.txt','.xml')]['New File Name'].values
    if len(File_new_name)==0:
        File_new_name=Files_names_data[Files_names_data['Old File Name']==Pre_Files[ii].replace('.txt','.pdf.xml')]['New File Name'].values
        if len(File_new_name)==0:
            continue
        File_new_name=File_new_name[0]
    else:
        File_new_name=File_new_name[0]
    Image_id=Actual_Files_names[Actual_Files_names['Imagename']==File_new_name.replace('.xml','.png')]['id'].values
    if len(Image_id)==0:
        Image_id=Actual_Files_names[Actual_Files_names['Imagename']==File_new_name.replace('.xml','.jpg')]['id'].values
    for g in prefile_data:
        _,score,x1,y1,x2,y2=g
        Data_predicted_json.append({"image_id":Image_id[0],"category_id":0,"bbox":[x1,y1,x2-x1,y2-y1],"score":score})

cocoGt=COCO(Ground_truth_json)
cocoDt=cocoGt.loadRes(Data_predicted_json)

cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.25s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.43s).
Accumulating evaluation results...
DONE (t=0.27s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.672
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.869
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.751
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.672
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.736
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet

In [21]:
Result=[]
for i in range(0,10):
    cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
    if i==0:
        i=0.1
    
    cocoEval.params.iouThrs=[i/10]
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()

    Result.append([i/10,np.round(cocoEval.stats[0],decimals=3),np.round(cocoEval.stats[8],decimals=3)])

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.86s).
Accumulating evaluation results...
DONE (t=0.15s).
 Average Precision  (AP) @[ IoU=0.01:0.01 | area=   all | maxDets=100 ] = 0.941
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.01:0.01 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.01:0.01 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.01:0.01 | area= large | maxDets=100 ] = 0.941
 Average Recall     (AR) @[ IoU=0.01:0.01 | area=   all | maxDets=  1 ] = 0.587
 Average Recall     (AR) @[ IoU=0.01:0.01 | area=   all | maxDets= 10 ] = 0.978
 Average Recall     (AR) @[ IoU=0.01:0.01 | area=   all | maxDets=100 ] = 0.978
 Average Recall     (AR) @[ IoU=0.01:0.01 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.01:0.01 | area=medium | maxDets

DONE (t=0.96s).
Accumulating evaluation results...
DONE (t=0.20s).
 Average Precision  (AP) @[ IoU=0.80:0.80 | area=   all | maxDets=100 ] = 0.690
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.80:0.80 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.80:0.80 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.80:0.80 | area= large | maxDets=100 ] = 0.690
 Average Recall     (AR) @[ IoU=0.80:0.80 | area=   all | maxDets=  1 ] = 0.500
 Average Recall     (AR) @[ IoU=0.80:0.80 | area=   all | maxDets= 10 ] = 0.757
 Average Recall     (AR) @[ IoU=0.80:0.80 | area=   all | maxDets=100 ] = 0.757
 Average Recall     (AR) @[ IoU=0.80:0.80 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.80:0.80 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.80:0.80 | area=

In [22]:
Result

[[0.01, 0.941, 0.978],
 [0.1, 0.94, 0.973],
 [0.2, 0.93, 0.962],
 [0.3, 0.911, 0.95],
 [0.4, 0.897, 0.939],
 [0.5, 0.869, 0.917],
 [0.6, 0.845, 0.896],
 [0.7, 0.794, 0.855],
 [0.8, 0.69, 0.757],
 [0.9, 0.413, 0.491]]

In [133]:
Results On> (Score:0.8)
[[0.01, 0.741, 0.748],
[0.1, 0.741, 0.748],
[0.2, 0.741, 0.747],
[0.3, 0.74, 0.746],
[0.4, 0.74, 0.743],
[0.5, 0.727, 0.734],
[0.6, 0.715, 0.725],
[0.7, 0.701, 0.712],
[0.8, 0.643, 0.668],
[0.9, 0.307, 0.407]]

[[0.01, 0.963, 0.971],
 [0.1, 0.954, 0.969],
 [0.2, 0.954, 0.965],
 [0.3, 0.943, 0.958],
 [0.4, 0.94, 0.951],
 [0.5, 0.915, 0.933],
 [0.6, 0.89, 0.912],
 [0.7, 0.845, 0.877],
 [0.8, 0.743, 0.788],
 [0.9, 0.332, 0.451]]

In [154]:
ALL_ious=list(cocoEval.ious.values())

In [162]:
ALL_ious0=[np.round(k,decimals=3)  for g in ALL_ious for h in g for k in h]

In [165]:
np.c_[np.unique(ALL_ious0,return_counts=True)]

array([[0.000e+00, 5.721e+03],
       [1.000e-03, 3.000e+00],
       [2.000e-03, 6.000e+00],
       ...,
       [9.890e-01, 1.000e+00],
       [9.900e-01, 1.000e+00],
       [9.910e-01, 1.000e+00]])

In [171]:
ALL_ious0=np.array(ALL_ious0)

In [177]:
np.unique(ALL_ious0[np.where(ALL_ious0<0.10)[0]],return_counts=True)

(array([0.   , 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008,
        0.009, 0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.017, 0.018,
        0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.027, 0.028, 0.029,
        0.031, 0.032, 0.033, 0.034, 0.035, 0.036, 0.037, 0.038, 0.039,
        0.04 , 0.041, 0.042, 0.043, 0.045, 0.046, 0.047, 0.048, 0.049,
        0.05 , 0.051, 0.052, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062,
        0.063, 0.064, 0.065, 0.066, 0.067, 0.071, 0.072, 0.073, 0.074,
        0.075, 0.076, 0.078, 0.079, 0.083, 0.084, 0.085, 0.086, 0.087,
        0.088, 0.089, 0.091, 0.092, 0.093, 0.095, 0.096, 0.097, 0.099]),
 array([5721,    3,    6,    3,    4,    4,    5,    2,    2,    1,    5,
           2,    3,    1,    3,    2,    4,    3,    4,    1,    2,    1,
           3,    3,    1,    1,    2,    1,    3,    3,    2,    2,    1,
           1,    2,    2,    2,    1,    2,    2,    3,    1,    2,    1,
           2,    2,    1,    1,    1,    1,    1,    2,    2,  

In [20]:
sum(Count_prediction)

7466

In [18]:
Pre_Files_dir='Predicted/'
Count_prediction=[]
for ii in range(len(Pre_Files)):
#     print(ii)
    with open(Pre_Files_dir+'/'+Pre_Files[ii],'r') as f:
        prefile_data0=f.read().split('\n')
    Count_prediction.append(len([g for g in prefile_data0 if g!='']))


#     prefile_data=[float(g.split()[1]) for g in prefile_data0 if g!='']
#     prefile_data0=np.delete(np.array(prefile_data0),np.where(np.array(prefile_data)<0.8)[0]).tolist()

#     with open('2'+Pre_Files_dir+'/'+Pre_Files[ii],'w') as f:
#         f.write('\n'.join(prefile_data0))